# 🎯 Glimpse3D - Master Pipeline

## Complete 2D Image → 3D Gaussian Splat Pipeline

This notebook runs the **entire Glimpse3D pipeline** end-to-end:

```
📷 Input Image
    ↓
🔷 TripoSR (0.5s) → Initial 3D Mesh → Gaussian Points
    ↓
🎨 SyncDreamer (2min) → 16 Consistent Multi-View Images
    ↓  
✨ SDXL Lightning + ControlNet → Enhanced Views
    ↓
🔮 gsplat Optimization → Refined Gaussians
    ↓
🔄 MVCRM → Multi-View Consistent Refinement
    ↓
🏆 Final 3D Gaussian Splat Output
```

## Requirements
- Google Colab with **T4 GPU** (free tier) or **A100** (faster)
- ~12GB VRAM peak usage
- ~30 minutes total runtime

---

## 🚀 Quick Start

1. Run all cells in order (Runtime → Run all)
2. Upload your image when prompted
3. Wait ~30 minutes for full pipeline
4. Download final results!

# Stage 0: Environment Setup

This stage:
1. Checks GPU availability and VRAM
2. Clones the Glimpse3D repository for `ai_modules` utilities
3. Installs all required dependencies
4. Creates the output directory structure

In [ ]:
# Check environment
import sys
import os

IN_COLAB = 'google.colab' in sys.modules
print(f"🖥️ Running in Colab: {IN_COLAB}")

if not IN_COLAB:
    print("⚠️ This notebook is designed for Google Colab!")
    print("   Some features may not work locally.")

# Check GPU
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

import torch
print(f"\n📦 PyTorch: {torch.__version__}")
print(f"🔥 CUDA: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    GPU_NAME = torch.cuda.get_device_name(0)
    GPU_VRAM = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🎮 GPU: {GPU_NAME}")
    print(f"💾 VRAM: {GPU_VRAM:.1f} GB")
    
    # Set batch sizes based on GPU
    if GPU_VRAM >= 40:  # A100
        BATCH_VIEW_NUM = 8
        MC_RESOLUTION = 384
        NUM_SAMPLES = 200000
    elif GPU_VRAM >= 15:  # T4/V100
        BATCH_VIEW_NUM = 4
        MC_RESOLUTION = 256
        NUM_SAMPLES = 100000
    else:  # Lower VRAM
        BATCH_VIEW_NUM = 2
        MC_RESOLUTION = 192
        NUM_SAMPLES = 50000
    
    print(f"\n⚙️ Settings: batch_view={BATCH_VIEW_NUM}, resolution={MC_RESOLUTION}, samples={NUM_SAMPLES}")
else:
    raise RuntimeError("❌ No GPU available! Enable GPU in Runtime → Change runtime type")

In [ ]:
# Clone Glimpse3D repository (for ai_modules utilities)
GLIMPSE3D_REPO = "/content/Glimpse-3D"

if not os.path.exists(GLIMPSE3D_REPO):
    print("📥 Cloning Glimpse3D repository...")
    !git clone https://github.com/varunaditya27/Glimpse3D.git {GLIMPSE3D_REPO}
else:
    print(f"✅ Glimpse3D repo already exists at {GLIMPSE3D_REPO}")

# Add to Python path for ai_modules
sys.path.insert(0, GLIMPSE3D_REPO)
print(f"✅ Added {GLIMPSE3D_REPO} to Python path")

In [ ]:
%%capture install_output
# Install all dependencies with PINNED VERSIONS for production stability
print("📦 Installing dependencies (this takes ~5 minutes)...")

# ⚠️ CRITICAL: Ensure PyTorch + CUDA version match
# Colab default torch may conflict - reinstall with exact versions
!pip uninstall torch torchvision torchaudio -y --quiet 2>/dev/null || true
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118 --quiet

# Core packages with pinned versions
!pip install transformers==4.40.0 diffusers==0.27.2 accelerate==0.28.0 huggingface_hub safetensors==0.4.2 --quiet
!pip install omegaconf==2.3.0 einops==0.7.0 pytorch-lightning==1.9.5 kornia==0.7.0 --quiet

# TripoSR dependencies
!pip install trimesh==4.2.0 rembg[gpu]==2.0.55 xatlas plyfile>=0.9.1 --quiet
!pip install git+https://github.com/tatsy/torchmcubes.git --quiet

# gsplat - must match torch 2.0.1+cu118
!pip install gsplat==1.2.0 --quiet

# SyncDreamer dependencies - PIN CLIP to known working commit
!pip install git+https://github.com/openai/CLIP.git@a1d071733d7111c9c014f024669f959182114e33 --quiet
!pip install taming-transformers-rom1504==0.0.6 --quiet

# Image processing
!pip install opencv-python-headless>=4.8.0 scikit-image>=0.21.0 imageio>=2.31.0 --quiet

# Depth estimation
!pip install timm --quiet

print("\n✅ All dependencies installed with pinned versions!")

In [ ]:
# Create directory structure
from pathlib import Path
import gc

WORK_DIR = Path("/content/glimpse3d_pipeline")
WORK_DIR.mkdir(exist_ok=True)

DIRS = {
    'input': WORK_DIR / 'input',
    'triposr': WORK_DIR / 'stage1_triposr',
    'syncdreamer': WORK_DIR / 'stage2_syncdreamer',
    'enhanced': WORK_DIR / 'stage3_enhanced',
    'gsplat': WORK_DIR / 'stage4_gsplat',
    'mvcrm': WORK_DIR / 'stage5_mvcrm',
    'output': WORK_DIR / 'final_output',
}

for name, path in DIRS.items():
    path.mkdir(exist_ok=True)
    print(f"📁 {name}: {path}")

def clear_gpu():
    """Aggressively clear GPU memory between stages."""
    # Multiple gc passes for thorough cleanup
    for _ in range(3):
        gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    print(f"🧹 GPU memory cleared. Allocated: {allocated:.2f} GB, Reserved: {reserved:.2f} GB")

def safe_del(obj_name, globals_dict):
    """Safely delete an object if it exists."""
    if obj_name in globals_dict and globals_dict[obj_name] is not None:
        del globals_dict[obj_name]
        
print("\n✅ Directory structure created!")

# Stage 1: Upload Input Image

In [ ]:
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

print("📤 Upload your image (JPG/PNG):")
uploaded = files.upload()

# Save uploaded file
INPUT_FILENAME = list(uploaded.keys())[0]
INPUT_PATH = DIRS['input'] / INPUT_FILENAME

with open(INPUT_PATH, 'wb') as f:
    f.write(list(uploaded.values())[0])

# Display
input_image = Image.open(INPUT_PATH)
plt.figure(figsize=(8, 8))
plt.imshow(input_image)
plt.title(f"Input: {INPUT_FILENAME} ({input_image.size[0]}x{input_image.size[1]})")
plt.axis('off')
plt.show()

print(f"\n✅ Saved to: {INPUT_PATH}")

# Stage 2: TripoSR - Initial 3D Reconstruction

**Input:** Single image  
**Output:** 3D mesh + Gaussian point cloud  
**Time:** ~30 seconds

In [ ]:
# Clone TripoSR
TRIPOSR_PATH = Path("/content/TripoSR")

if not TRIPOSR_PATH.exists():
    print("📥 Cloning TripoSR...")
    !git clone https://github.com/VAST-AI-Research/TripoSR.git {TRIPOSR_PATH}

sys.path.insert(0, str(TRIPOSR_PATH))
os.chdir(TRIPOSR_PATH)
print(f"✅ TripoSR ready at {TRIPOSR_PATH}")

In [ ]:
import time
from tsr.system import TSR
from tsr.utils import remove_background, resize_foreground
import rembg

print("\n" + "="*60)
print("🔷 STAGE 2: TripoSR 3D Reconstruction")
print("="*60)

device = "cuda:0"

# Load model
print("\n📥 Loading TripoSR model...")
triposr_model = TSR.from_pretrained(
    "stabilityai/TripoSR",
    config_name="config.yaml",
    weight_name="model.ckpt",
)
triposr_model.renderer.set_chunk_size(8192)
triposr_model.to(device)
print("✅ Model loaded!")

# Preprocess image
print("\n🔧 Preprocessing image...")
input_img = Image.open(INPUT_PATH)
rembg_session = rembg.new_session()
processed_img = remove_background(input_img, rembg_session)
processed_img = resize_foreground(processed_img, 0.85)

# Convert to RGB
img_np = np.array(processed_img).astype(np.float32) / 255.0
img_np = img_np[:, :, :3] * img_np[:, :, 3:4] + (1 - img_np[:, :, 3:4]) * 0.5
processed_img = Image.fromarray((img_np * 255.0).astype(np.uint8))
processed_img.save(DIRS['triposr'] / "processed_input.png")

# Run inference
print("\n🚀 Running TripoSR...")
start_time = time.time()

with torch.no_grad():
    scene_codes = triposr_model([processed_img], device=device)
    meshes = triposr_model.extract_mesh(scene_codes, has_vertex_color=True, resolution=MC_RESOLUTION)

mesh = meshes[0]
elapsed = time.time() - start_time

print(f"\n✅ Mesh generated in {elapsed:.2f}s")
print(f"   Vertices: {len(mesh.vertices):,}")
print(f"   Faces: {len(mesh.faces):,}")

# Save mesh
mesh.export(str(DIRS['triposr'] / "mesh.obj"))
mesh.export(str(DIRS['triposr'] / "mesh.glb"))
print(f"\n📁 Saved mesh to {DIRS['triposr']}")

In [ ]:
# Convert mesh to Gaussian PLY
from plyfile import PlyData, PlyElement

def mesh_to_gaussian_ply(mesh, output_path, num_samples=100000):
    """Convert mesh to Gaussian Splat format."""
    print(f"\n🔄 Sampling {num_samples:,} points...")
    
    points, face_indices = mesh.sample(num_samples, return_index=True)
    
    if mesh.visual.vertex_colors is not None:
        face_vertices = mesh.faces[face_indices]
        vertex_colors = mesh.visual.vertex_colors[:, :3] / 255.0
        colors = vertex_colors[face_vertices].mean(axis=1)
    else:
        colors = np.ones((num_samples, 3)) * 0.5
    
    num_points = len(points)
    xyz = points.astype(np.float32)
    
    C0 = 0.28209479177387814
    features_dc = ((colors - 0.5) / C0).astype(np.float32)
    features_rest = np.zeros((num_points, 45), dtype=np.float32)
    opacities = np.ones((num_points, 1), dtype=np.float32) * 2.2
    scales = np.ones((num_points, 3), dtype=np.float32) * (-4.6)
    # ✅ gsplat uses wxyz quaternion convention: rot_0=w, rot_1=x, rot_2=y, rot_3=z
    rotations = np.zeros((num_points, 4), dtype=np.float32)
    rotations[:, 0] = 1.0  # w=1 (identity rotation in wxyz format)
    
    dtype_full = [
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
        ('f_dc_0', 'f4'), ('f_dc_1', 'f4'), ('f_dc_2', 'f4'),
    ]
    for i in range(45):
        dtype_full.append((f'f_rest_{i}', 'f4'))
    dtype_full.extend([
        ('opacity', 'f4'),
        ('scale_0', 'f4'), ('scale_1', 'f4'), ('scale_2', 'f4'),
        ('rot_0', 'f4'), ('rot_1', 'f4'), ('rot_2', 'f4'), ('rot_3', 'f4'),
    ])
    
    elements = np.zeros(num_points, dtype=dtype_full)
    elements['x'] = xyz[:, 0]
    elements['y'] = xyz[:, 1]
    elements['z'] = xyz[:, 2]
    elements['f_dc_0'] = features_dc[:, 0]
    elements['f_dc_1'] = features_dc[:, 1]
    elements['f_dc_2'] = features_dc[:, 2]
    for i in range(45):
        elements[f'f_rest_{i}'] = features_rest[:, i]
    elements['opacity'] = opacities[:, 0]
    elements['scale_0'] = scales[:, 0]
    elements['scale_1'] = scales[:, 1]
    elements['scale_2'] = scales[:, 2]
    elements['rot_0'] = rotations[:, 0]
    elements['rot_1'] = rotations[:, 1]
    elements['rot_2'] = rotations[:, 2]
    elements['rot_3'] = rotations[:, 3]
    
    el = PlyElement.describe(elements, 'vertex')
    PlyData([el]).write(output_path)
    print(f"✅ Saved: {output_path}")

INITIAL_PLY_PATH = DIRS['triposr'] / "initial_gaussian.ply"
mesh_to_gaussian_ply(mesh, str(INITIAL_PLY_PATH), num_samples=NUM_SAMPLES)

# Cleanup TripoSR
del triposr_model, mesh, scene_codes
clear_gpu()

# Stage 3: SyncDreamer - Multi-View Generation

**Input:** Processed image  
**Output:** 16 consistent multi-view images  
**Time:** ~2-3 minutes

In [ ]:
print("\n" + "="*60)
print("🎨 STAGE 3: SyncDreamer Multi-View Generation")
print("="*60)

# Clone SyncDreamer
SYNCDREAMER_PATH = Path("/content/SyncDreamer")

if not SYNCDREAMER_PATH.exists():
    print("📥 Cloning SyncDreamer...")
    !git clone https://github.com/liuyuan-pal/SyncDreamer.git {SYNCDREAMER_PATH}

# Download checkpoints
CKPT_DIR = SYNCDREAMER_PATH / "ckpt"
CKPT_DIR.mkdir(exist_ok=True)

!apt -y install -qq aria2

CHECKPOINTS = {
    "syncdreamer-pretrain.ckpt": "https://huggingface.co/camenduru/SyncDreamer/resolve/main/syncdreamer-pretrain.ckpt",
    "ViT-L-14.pt": "https://huggingface.co/camenduru/SyncDreamer/resolve/main/ViT-L-14.pt"
}

for fname, url in CHECKPOINTS.items():
    fpath = CKPT_DIR / fname
    if not fpath.exists():
        print(f"📥 Downloading {fname}...")
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d "{CKPT_DIR}" -o "{fname}"
    else:
        print(f"✅ {fname} exists")

sys.path.insert(0, str(SYNCDREAMER_PATH))
os.chdir(SYNCDREAMER_PATH)

In [ ]:
from omegaconf import OmegaConf
from ldm.util import instantiate_from_config

# Load SyncDreamer model
print("\n📥 Loading SyncDreamer model...")

config_path = SYNCDREAMER_PATH / "configs" / "syncdreamer.yaml"
config = OmegaConf.load(config_path)

# Instantiate model from config
syncdreamer_model = instantiate_from_config(config.model)

# Load pretrained weights
ckpt_path = CKPT_DIR / "syncdreamer-pretrain.ckpt"
print(f"   Loading checkpoint: {ckpt_path}")

checkpoint = torch.load(ckpt_path, map_location="cpu")
state_dict = checkpoint["state_dict"] if "state_dict" in checkpoint else checkpoint

# Load with strict=False to handle any minor mismatches
missing, unexpected = syncdreamer_model.load_state_dict(state_dict, strict=False)
if missing:
    print(f"   ⚠️ Missing keys: {len(missing)}")
if unexpected:
    print(f"   ⚠️ Unexpected keys: {len(unexpected)}")

syncdreamer_model = syncdreamer_model.cuda().eval()

# Verify model is ready
print(f"✅ SyncDreamer loaded!")
print(f"   Model type: {type(syncdreamer_model).__name__}")

In [ ]:
from ldm.models.diffusion.sync_dreamer import SyncDDIMSampler
from ldm.util import prepare_inputs  # CRITICAL: Use official data preparation

# ✅ FIXED: Camera configuration MUST match SyncDreamer training data
# SyncDreamer generates 16 views at FIXED 30° elevation, azimuths spaced 22.5° apart
ELEVATIONS = [30.0] * 16  # All 16 views at 30° elevation
AZIMUTHS = [i * 22.5 for i in range(16)]  # 0°, 22.5°, 45°, ..., 337.5°
RADIUS = 1.5

# ✅ FIXED: Use official prepare_inputs function for proper data preparation
# This handles alpha channel, CLIP embedding, and proper normalization
processed_path = DIRS['triposr'] / "processed_input.png"

INPUT_ELEVATION = 30.0  # Assume front view at 30 degrees
CROP_SIZE = 200         # Crop foreground to this size

print(f"📸 Preparing input: {processed_path}")
print(f"   Input elevation: {INPUT_ELEVATION}°")
print(f"   Crop size: {CROP_SIZE}")

# Use official SyncDreamer data preparation
data = prepare_inputs(str(processed_path), INPUT_ELEVATION, CROP_SIZE)

# Move to GPU and add batch dimension
for k, v in data.items():
    data[k] = v.unsqueeze(0).cuda()
    print(f"   {k}: {data[k].shape}")

print(f"\n✅ Input prepared using official prepare_inputs()")

In [ ]:
# Run SyncDreamer inference
print("\n🚀 Running SyncDreamer (this takes ~2-3 minutes)...")
start_time = time.time()

# Settings
SAMPLE_STEPS = 50
CFG_SCALE = 2.0

sampler = SyncDDIMSampler(syncdreamer_model, SAMPLE_STEPS)

try:
    with torch.no_grad():
        # ✅ FIXED: Data already prepared correctly by prepare_inputs()
        # Run synchronized multi-view generation
        x_sample = syncdreamer_model.sample(
            sampler, 
            data, 
            CFG_SCALE, 
            BATCH_VIEW_NUM
        )
        # x_sample shape: (B, N, C, H, W) where N=16 views
        
except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print("⚠️ OOM Error! Reducing batch size and retrying...")
        clear_gpu()
        BATCH_VIEW_NUM = max(1, BATCH_VIEW_NUM // 2)
        print(f"   New BATCH_VIEW_NUM: {BATCH_VIEW_NUM}")
        
        sampler = SyncDDIMSampler(syncdreamer_model, SAMPLE_STEPS)
        with torch.no_grad():
            x_sample = syncdreamer_model.sample(sampler, data, CFG_SCALE, BATCH_VIEW_NUM)
    else:
        raise

elapsed = time.time() - start_time
print(f"\n✅ SyncDreamer completed in {elapsed/60:.1f} minutes")
print(f"   Output shape: {x_sample.shape}")

In [ ]:
# Save multi-view images
print("\n💾 Saving multi-view images...")

# Convert samples to images: [-1,1] -> [0,1]
samples = (x_sample.clamp(-1, 1) + 1) / 2

syncdreamer_views = []

B, N, C, H, W = samples.shape
print(f"   Processing {N} views at {H}x{W}")

for i in range(N):
    img_tensor = samples[0, i]  # (C, H, W)
    img_np = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
    img_pil = Image.fromarray(img_np)
    
    # Save individual view
    elev = int(ELEVATIONS[i])
    azim = int(AZIMUTHS[i])
    save_path = DIRS['syncdreamer'] / f"view_{i:02d}_e{elev}_a{azim}.png"
    img_pil.save(save_path)
    syncdreamer_views.append(img_pil)

print(f"✅ Saved {len(syncdreamer_views)} views to {DIRS['syncdreamer']}")

# Display grid
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
for i, ax in enumerate(axes.flat):
    ax.imshow(syncdreamer_views[i])
    ax.set_title(f"E={int(ELEVATIONS[i])}° A={int(AZIMUTHS[i])}°", fontsize=8)
    ax.axis('off')
plt.suptitle("SyncDreamer: 16 Multi-View Images", fontsize=14)
plt.tight_layout()
plt.savefig(DIRS['syncdreamer'] / "grid.png", dpi=150)
plt.show()

# Cleanup SyncDreamer to free VRAM
del syncdreamer_model, sampler, x_sample, samples
clear_gpu()

# Stage 4: SDXL Enhancement (Optional)

**Input:** Multi-view images  
**Output:** Enhanced multi-view images  
**Time:** ~1 minute per image

Skip this stage if you want faster results.

In [ ]:
SKIP_ENHANCEMENT = False  # Set to True to skip this stage

if not SKIP_ENHANCEMENT:
    print("\n" + "="*60)
    print("✨ STAGE 4: SDXL Lightning Enhancement")
    print("="*60)
    
    from diffusers import StableDiffusionXLImg2ImgPipeline, AutoencoderKL, UNet2DConditionModel, EulerDiscreteScheduler
    from huggingface_hub import hf_hub_download
    from safetensors.torch import load_file
    
    # ✅ FIXED: Load SDXL Lightning (UNet checkpoint method - per official ByteDance docs)
    print("\n📥 Loading SDXL Lightning...")
    print("   Using UNet checkpoint method (recommended by ByteDance)")
    
    base_model = "stabilityai/stable-diffusion-xl-base-1.0"
    repo = "ByteDance/SDXL-Lightning"
    
    try:
        # Load VAE with fp16 fix
        vae = AutoencoderKL.from_pretrained(
            "madebyollin/sdxl-vae-fp16-fix",
            torch_dtype=torch.float16
        )
        
        # Load UNet from base model config, then apply Lightning weights
        unet = UNet2DConditionModel.from_config(base_model, subfolder="unet")
        unet = unet.to(dtype=torch.float16)
        
        # Download and load Lightning UNet weights (4-step version)
        print("   Downloading Lightning UNet weights...")
        unet_path = hf_hub_download(repo, "sdxl_lightning_4step_unet.safetensors")
        unet.load_state_dict(load_file(unet_path, device="cuda"), strict=True)
        
        # Create pipeline
        pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
            base_model,
            unet=unet,
            vae=vae,
            torch_dtype=torch.float16,
            variant="fp16",
        )
        
        # ✅ CRITICAL: Use correct scheduler for Lightning
        # Lightning was trained with "trailing" timestep spacing
        pipe.scheduler = EulerDiscreteScheduler.from_config(
            pipe.scheduler.config, 
            timestep_spacing="trailing"
        )
        
        pipe = pipe.to("cuda")
        print("✅ SDXL Lightning loaded!")
        print("   ⚠️ Remember: guidance_scale MUST be 0 for Lightning")
        
    except Exception as e:
        print(f"❌ Failed to load SDXL Lightning: {e}")
        print("   Falling back to skip enhancement")
        SKIP_ENHANCEMENT = True
else:
    print("⏭️ Skipping enhancement stage")

In [ ]:
if not SKIP_ENHANCEMENT:
    # Enhance select views (not all 16 to save time)
    VIEWS_TO_ENHANCE = [0, 2, 4, 6, 8, 10, 12, 14]  # Every other view
    
    print(f"\n🚀 Enhancing {len(VIEWS_TO_ENHANCE)} views...")
    
    enhanced_views = syncdreamer_views.copy()  # Start with original
    
    prompt = "highly detailed 3D render, professional studio lighting, sharp textures, photorealistic, 8k quality"
    negative_prompt = "blurry, low quality, artifacts, noise, watermark, text"
    
    for i, view_idx in enumerate(VIEWS_TO_ENHANCE):
        print(f"  Enhancing view {view_idx} ({i+1}/{len(VIEWS_TO_ENHANCE)})...")
        
        # Resize input for SDXL (works best at 512-1024)
        input_img = syncdreamer_views[view_idx].resize((512, 512), Image.LANCZOS)
        
        with torch.no_grad():
            result = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                image=input_img,
                strength=0.35,  # Lower = preserve more original structure
                num_inference_steps=4,  # Lightning uses 4 steps
                guidance_scale=0,  # Lightning uses CFG=0
            ).images[0]
        
        # Resize back to match SyncDreamer output size
        result_resized = result.resize((256, 256), Image.LANCZOS)
        enhanced_views[view_idx] = result_resized
        result.save(DIRS['enhanced'] / f"enhanced_{view_idx:02d}.png")
    
    print(f"\n✅ Enhanced views saved to {DIRS['enhanced']}")
    
    # Cleanup
    del pipe, vae, unet
    clear_gpu()
else:
    enhanced_views = syncdreamer_views
    print("Using original SyncDreamer views")

# Stage 5: gsplat Optimization

**Input:** Initial Gaussian PLY + Multi-view images  
**Output:** Optimized Gaussian Splats  
**Time:** ~5 minutes

In [ ]:
print("\n" + "="*60)
print("🔮 STAGE 5: gsplat Optimization")
print("="*60)

import torch.nn as nn
from gsplat import rasterization
import math

device = torch.device("cuda:0")

# Image size for rendering (matches SyncDreamer output)
IMAGE_SIZE = 256

# Load initial Gaussians from PLY
def load_gaussian_ply(path):
    """Load Gaussian parameters from PLY file."""
    plydata = PlyData.read(path)
    vertex = plydata['vertex']
    
    xyz = np.stack([vertex['x'], vertex['y'], vertex['z']], axis=-1)
    f_dc = np.stack([vertex['f_dc_0'], vertex['f_dc_1'], vertex['f_dc_2']], axis=-1)
    
    # Load f_rest if present
    f_rest_names = [f'f_rest_{i}' for i in range(45)]
    available_f_rest = [name for name in f_rest_names if name in vertex.data.dtype.names]
    if available_f_rest:
        f_rest = np.stack([vertex[name] for name in available_f_rest], axis=-1)
    else:
        f_rest = np.zeros((len(xyz), 45), dtype=np.float32)
    
    # ✅ FIXED: Ensure opacity is 1D (N,) - gsplat requires this shape
    opacity = np.asarray(vertex['opacity'], dtype=np.float32).flatten()
    scales = np.stack([vertex['scale_0'], vertex['scale_1'], vertex['scale_2']], axis=-1)
    rotations = np.stack([vertex['rot_0'], vertex['rot_1'], vertex['rot_2'], vertex['rot_3']], axis=-1)
    
    return {
        'xyz': torch.tensor(xyz, dtype=torch.float32),
        'f_dc': torch.tensor(f_dc, dtype=torch.float32),
        'f_rest': torch.tensor(f_rest, dtype=torch.float32),
        'opacity': torch.tensor(opacity, dtype=torch.float32),  # Shape: (N,)
        'scales': torch.tensor(scales, dtype=torch.float32),
        'rotations': torch.tensor(rotations, dtype=torch.float32),
    }

gaussians = load_gaussian_ply(str(INITIAL_PLY_PATH))
print(f"✅ Loaded {len(gaussians['xyz']):,} Gaussians")

In [ ]:
class GaussianModel(nn.Module):
    def __init__(self, gaussians):
        super().__init__()
        self.xyz = nn.Parameter(gaussians['xyz'].clone())
        self.f_dc = nn.Parameter(gaussians['f_dc'].clone())
        self.f_rest = nn.Parameter(gaussians['f_rest'].clone())
        # ✅ FIXED: Ensure opacity is 1D (N,) - gsplat requires this shape
        opacity_tensor = gaussians['opacity'].clone()
        if opacity_tensor.dim() > 1:
            opacity_tensor = opacity_tensor.squeeze(-1)
        self.opacity_raw = nn.Parameter(opacity_tensor)
        self.scales_raw = nn.Parameter(gaussians['scales'].clone())
        self.rotations = nn.Parameter(gaussians['rotations'].clone())
        
    @property
    def opacity(self):
        # ✅ FIXED: Returns (N,) shape - required by gsplat.rasterization()
        return torch.sigmoid(self.opacity_raw)
    
    @property
    def scales(self):
        return torch.exp(self.scales_raw)
    
    def get_colors(self):
        C0 = 0.28209479177387814
        return 0.5 + C0 * self.f_dc
    
    def forward(self):
        return {
            'xyz': self.xyz,
            'colors': self.get_colors(),
            'opacity': self.opacity,
            'scales': self.scales,
            'rotations': self.rotations / (self.rotations.norm(dim=-1, keepdim=True) + 1e-8),
        }

model = GaussianModel(gaussians).to(device)
print(f"✅ Model: {sum(p.numel() for p in model.parameters()):,} parameters")

In [ ]:
# Camera system matching SyncDreamer conventions
# SyncDreamer uses: Y-up, camera looks at origin, radius ~1.5

def create_camera_pose(elevation_deg, azimuth_deg, radius=1.5):
    """Create world-to-camera matrix for given elevation and azimuth."""
    elev = math.radians(elevation_deg)
    azim = math.radians(azimuth_deg)
    
    # Camera position in spherical coordinates (Y-up convention)
    x = radius * math.cos(elev) * math.sin(azim)
    y = radius * math.sin(elev)
    z = radius * math.cos(elev) * math.cos(azim)
    
    cam_pos = np.array([x, y, z])
    look_at = np.array([0, 0, 0])
    up = np.array([0, 1, 0])  # Y-up
    
    # Construct camera basis
    forward = look_at - cam_pos
    forward = forward / (np.linalg.norm(forward) + 1e-8)
    right = np.cross(forward, up)
    right = right / (np.linalg.norm(right) + 1e-8)
    up_new = np.cross(right, forward)
    
    # World-to-camera transformation
    # R rotates world to camera, t translates
    w2c = np.eye(4, dtype=np.float32)
    w2c[0, :3] = right
    w2c[1, :3] = up_new
    w2c[2, :3] = -forward  # Camera looks along -Z
    w2c[:3, 3] = -w2c[:3, :3] @ cam_pos
    
    return w2c

def get_intrinsics(fov_deg=49.1, image_size=256):
    """Get camera intrinsics matrix. FOV ~49.1 matches SyncDreamer."""
    fov_rad = math.radians(fov_deg)
    focal = image_size / (2 * math.tan(fov_rad / 2))
    
    K = np.array([
        [focal, 0, image_size / 2],
        [0, focal, image_size / 2],
        [0, 0, 1]
    ], dtype=np.float32)
    return K

# Pre-compute all camera poses (using SyncDreamer camera parameters)
camera_poses = [create_camera_pose(e, a, radius=RADIUS) for e, a in zip(ELEVATIONS, AZIMUTHS)]
intrinsics = get_intrinsics(fov_deg=49.1, image_size=IMAGE_SIZE)

print(f"✅ Created {len(camera_poses)} camera poses")
print(f"   Image size: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"   Intrinsics: focal={intrinsics[0,0]:.1f}, center=({intrinsics[0,2]:.0f}, {intrinsics[1,2]:.0f})")

In [ ]:
def render_gaussians(model, w2c, K, image_size):
    """Render Gaussian splats from a camera viewpoint."""
    params = model()
    
    viewmat = torch.tensor(w2c, dtype=torch.float32, device=device)
    K_tensor = torch.tensor(K, dtype=torch.float32, device=device)
    
    try:
        render_colors, render_alphas, info = rasterization(
            means=params['xyz'],
            quats=params['rotations'],
            scales=params['scales'],
            opacities=params['opacity'],
            colors=params['colors'],
            viewmats=viewmat.unsqueeze(0),
            Ks=K_tensor.unsqueeze(0),
            width=image_size,
            height=image_size,
            packed=False,
            render_mode="RGB",
        )
        return render_colors[0], render_alphas[0]
    except Exception as e:
        print(f"Render error: {e}")
        # Return empty image on error
        return torch.zeros(image_size, image_size, 3, device=device), torch.zeros(image_size, image_size, 1, device=device)

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

# Prepare target images as tensors
target_tensors = []
for img in enhanced_views:
    img_resized = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.LANCZOS)
    img_tensor = torch.tensor(np.array(img_resized) / 255.0, dtype=torch.float32, device=device)
    target_tensors.append(img_tensor)

print(f"✅ Prepared {len(target_tensors)} target images at {IMAGE_SIZE}x{IMAGE_SIZE}")

# Optimizer with appropriate learning rates
optimizer = torch.optim.Adam([
    {'params': model.xyz, 'lr': 1e-4, 'name': 'xyz'},
    {'params': model.f_dc, 'lr': 2.5e-3, 'name': 'f_dc'},
    {'params': model.f_rest, 'lr': 2.5e-3 / 20, 'name': 'f_rest'},
    {'params': model.opacity_raw, 'lr': 0.05, 'name': 'opacity'},
    {'params': model.scales_raw, 'lr': 5e-3, 'name': 'scales'},
    {'params': model.rotations, 'lr': 1e-3, 'name': 'rotations'},
])

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.995)

# Training loop
NUM_ITERATIONS = 1000
losses = []

print(f"\n🚀 Starting optimization for {NUM_ITERATIONS} iterations...")
pbar = tqdm(range(NUM_ITERATIONS))

for iteration in pbar:
    optimizer.zero_grad()
    
    # Sample random view
    view_idx = np.random.randint(0, 16)
    w2c = camera_poses[view_idx]
    target = target_tensors[view_idx]
    
    # Render
    rendered, alpha = render_gaussians(model, w2c, intrinsics, IMAGE_SIZE)
    
    # L1 + SSIM loss (L1 is more stable)
    l1_loss = F.l1_loss(rendered, target)
    loss = l1_loss
    
    loss.backward()
    
    # Gradient clipping for stability
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    optimizer.step()
    scheduler.step()
    
    losses.append(loss.item())
    
    if iteration % 100 == 0:
        pbar.set_postfix({'loss': f'{loss.item():.4f}', 'view': view_idx})

print(f"\n✅ Optimization complete!")
print(f"   Initial loss: {losses[0]:.4f}")
print(f"   Final loss: {losses[-1]:.4f}")

# Plot loss curve
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('gsplat Optimization Loss')
plt.grid(True)
plt.savefig(DIRS['gsplat'] / "loss_curve.png")
plt.show()

In [ ]:
# Save optimized model
def save_gaussian_ply(model, output_path):
    with torch.no_grad():
        params = model()
        xyz = params['xyz'].cpu().numpy()
        colors = model.f_dc.cpu().numpy()
        f_rest = model.f_rest.cpu().numpy()
        opacity = model.opacity_raw.cpu().numpy()
        scales = model.scales_raw.cpu().numpy()
        rotations = params['rotations'].cpu().numpy()
        
    num_points = len(xyz)
    dtype_full = [('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
                  ('f_dc_0', 'f4'), ('f_dc_1', 'f4'), ('f_dc_2', 'f4')]
    for i in range(f_rest.shape[1]):
        dtype_full.append((f'f_rest_{i}', 'f4'))
    dtype_full.extend([('opacity', 'f4'),
                       ('scale_0', 'f4'), ('scale_1', 'f4'), ('scale_2', 'f4'),
                       ('rot_0', 'f4'), ('rot_1', 'f4'), ('rot_2', 'f4'), ('rot_3', 'f4')])
    
    elements = np.zeros(num_points, dtype=dtype_full)
    elements['x'] = xyz[:, 0]
    elements['y'] = xyz[:, 1]
    elements['z'] = xyz[:, 2]
    elements['f_dc_0'] = colors[:, 0]
    elements['f_dc_1'] = colors[:, 1]
    elements['f_dc_2'] = colors[:, 2]
    for i in range(f_rest.shape[1]):
        elements[f'f_rest_{i}'] = f_rest[:, i]
    elements['opacity'] = opacity
    elements['scale_0'] = scales[:, 0]
    elements['scale_1'] = scales[:, 1]
    elements['scale_2'] = scales[:, 2]
    elements['rot_0'] = rotations[:, 0]
    elements['rot_1'] = rotations[:, 1]
    elements['rot_2'] = rotations[:, 2]
    elements['rot_3'] = rotations[:, 3]
    
    el = PlyElement.describe(elements, 'vertex')
    PlyData([el]).write(output_path)

OPTIMIZED_PLY_PATH = DIRS['gsplat'] / "optimized_gaussian.ply"
save_gaussian_ply(model, str(OPTIMIZED_PLY_PATH))
print(f"✅ Saved optimized Gaussians: {OPTIMIZED_PLY_PATH}")

# Stage 6: Generate Final Outputs

In [ ]:
print("\n" + "="*60)
print("🏆 FINAL OUTPUT GENERATION")
print("="*60)

import imageio

# Generate 360° video at higher resolution
VIDEO_SIZE = 512
video_intrinsics = get_intrinsics(fov_deg=49.1, image_size=VIDEO_SIZE)

print("\n🎬 Rendering 360° turntable video...")
video_frames = []

with torch.no_grad():
    for azim in tqdm(np.linspace(0, 360, 120, endpoint=False)):
        w2c = create_camera_pose(20.0, azim, radius=RADIUS)  # Slightly lower angle for nicer view
        rgb, _ = render_gaussians(model, w2c, video_intrinsics, VIDEO_SIZE)
        frame = (rgb.cpu().numpy().clip(0, 1) * 255).astype(np.uint8)
        video_frames.append(frame)

video_path = DIRS['output'] / "glimpse3d_360.mp4"
imageio.mimsave(str(video_path), video_frames, fps=30)
print(f"✅ Video saved: {video_path}")

In [ ]:
# Copy final files
import shutil

# Copy optimized PLY
final_ply = DIRS['output'] / "final_gaussian.ply"
shutil.copy(OPTIMIZED_PLY_PATH, final_ply)

# Copy mesh
shutil.copy(DIRS['triposr'] / "mesh.glb", DIRS['output'] / "initial_mesh.glb")
shutil.copy(DIRS['triposr'] / "mesh.obj", DIRS['output'] / "initial_mesh.obj")

# Copy best views
for i in [0, 4, 8, 12]:
    shutil.copy(
        DIRS['syncdreamer'] / f"view_{i:02d}_e{int(ELEVATIONS[i])}_a{int(AZIMUTHS[i])}.png",
        DIRS['output'] / f"view_{i:02d}.png"
    )

print("\n📁 Final output files:")
for f in sorted(DIRS['output'].iterdir()):
    size_mb = f.stat().st_size / 1024 / 1024
    print(f"  {f.name} ({size_mb:.1f} MB)")

In [ ]:
# Display video
from IPython.display import HTML
from base64 import b64encode

mp4 = open(video_path, 'rb').read()
data_url = f"data:video/mp4;base64,{b64encode(mp4).decode()}"
HTML(f'''
<h3>🏆 Glimpse3D Result</h3>
<video width="600" controls autoplay loop>
    <source src="{data_url}" type="video/mp4">
</video>
''')

# 📥 Download All Results

In [ ]:
from google.colab import files

# Create final ZIP
output_zip = str(WORK_DIR / "glimpse3d_complete_output")
shutil.make_archive(output_zip, 'zip', DIRS['output'])

print("📥 Downloading Glimpse3D results...")
files.download(f"{output_zip}.zip")

print("\n" + "="*60)
print("✅ GLIMPSE3D PIPELINE COMPLETE!")
print("="*60)
print(f"\nDownloaded: glimpse3d_complete_output.zip")
print("\nContents:")
print("  - final_gaussian.ply   : Optimized Gaussian Splats")
print("  - initial_mesh.glb/obj : TripoSR mesh")
print("  - glimpse3d_360.mp4    : 360° turntable video")
print("  - view_*.png           : Multi-view images")

---

## 🎉 Pipeline Complete!

You now have:
1. **final_gaussian.ply** - View in any Gaussian Splat viewer
2. **initial_mesh.glb** - View in 3D viewers like Blender, online GLB viewers
3. **glimpse3d_360.mp4** - Share as video

### Recommended Viewers
- **Gaussian Splats**: [SuperSplat](https://playcanvas.com/supersplat/editor), [Luma AI Viewer](https://lumalabs.ai/)
- **GLB Mesh**: [glTF Viewer](https://gltf-viewer.donmccurdy.com/), Blender

### Tips for Better Results
1. Use high-quality input images with clean backgrounds
2. Objects should be centered and fill ~80% of the frame
3. Avoid reflective or transparent surfaces
4. Run more gsplat iterations (2000+) for higher quality